# Lending protocols and short/long leverage data

Load lending market data for a specific chain (Polygon in this example)
and diplay status of the lending market.

In this specific example we download interest rates and prices
for all assets on Aave v3 on Polygon.

As this notebook always uses latest data, it does not cache any downloaded datasets
and may be slow to run depending on the speed of your Internet.

## Set up cient and API keys to download dataset

In [1]:
import datetime

import pandas as pd
from tradingstrategy.client import Client

# You will be interactively prompted to register an API key if you do not have one yet
client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


## Download datasets

We use a shortcut method `load_trading_and_lending_data` to grab everything we need.

- This function can download current data
- This function can download data for historical time periods

In [2]:
from tradingstrategy.chain import ChainId
from tradeexecutor.strategy.universe_model import UniverseOptions
from tradeexecutor.strategy.execution_context import ExecutionMode, ExecutionContext
from tradeexecutor.strategy.trading_strategy_universe import load_trading_and_lending_data, TradingStrategyUniverse

# Download 30 days historical from today
lookback = datetime.timedelta(days=30)

# Even within Uniswap v3, there will be multiple trading pairs
# for each lending market asset, we need to specify
# which ones we are interested in.
# We pick USDC because it's most used stablecoin in DeFi
# environment.
#
# Even with USDC some assets are quoted
# with different Uniswap v3 fee tiers,
# like 30 BPS and 5 BPS.
# We deal with that later.
quote_token = "USDC"

# Load all trading and lending data on Polygon
# for all lending markets on a relevant time period
dataset = load_trading_and_lending_data(
    client,
    execution_context=ExecutionContext(mode=ExecutionMode.data_research),
    universe_options=UniverseOptions(history_period=lookback),
    # Ask for all Polygon data
    chain_id=ChainId.polygon,
    # We limit ourselves to price feeds on Uniswap v3 and Quickswap on Polygon,
    # as there are multiple small or dead DEXes on Polygon
    # which also have price feeds but not interesting liquidity
    exchange_slugs={"uniswap-v3", "quickswap"},

    reserve_asset_symbols={quote_token},

    # We want to display stablecoin-stablecoin
    # trading pairs
    volatile_only=False,
)

# Construct a trading universe that covers all
# chains, DEXes, trading pairs and lending pools we specified
strategy_universe = TradingStrategyUniverse.create_from_dataset(dataset)
data_universe = strategy_universe.data_universe

print(f"We have\n"
      f"- {data_universe.pairs.get_count()} trading pairs\n"
      f"- {data_universe.lending_reserves.get_count()} lending reserves\n"
      f"- {data_universe.candles.get_candle_count()} price candles ({data_universe.time_bucket.value})")

We have
- 19 trading pairs
- 20 lending reserves
- 372 price candles (1d)


## Display available trading pair and lending asset data

Display the status of the trading and lending markets.

We use `get_pair_by_human_description()` to find the lowest fee USDC trading pair
for each lending reserve asset.

We determine if a trading pair is long/short leveraged tradeable by
checking if it has enough trading volume on Polygon on any (indexed) DEX.
The threshold is either
- USD 100k monthly trading volume (no liquidity available)
- USD 100k liquidity


In [3]:
from IPython.display import HTML

from tradingstrategy.candle import CandleSampleUnavailable
from tradingstrategy.stablecoin import is_stablecoin_like
from tradingstrategy.pair import PairNotFoundError
from tradingstrategy.utils.jupyter import format_links_for_html_output

rows = []

now = datetime.datetime.utcnow()  # We always operate on naive UTC timesetamps
past = now - datetime.timedelta(days=29)

# We are still happy if we get 2 days old price
# (Some low liquidity pairs may see very few trades)
price_delay_tolerance = pd.Timedelta(days=2)

for reserve in data_universe.lending_reserves.iterate_reserves():

    stablecoin = is_stablecoin_like(reserve.asset_symbol)

    lending_link = reserve.get_link()

    supply_apr_now, lag = data_universe.lending_candles.supply_apr.get_single_rate(
        reserve,
        now,
        data_lag_tolerance=price_delay_tolerance,
    )

    borrow_apr_now, lag = data_universe.lending_candles.variable_borrow_apr.get_single_rate(
        reserve,
        now,
        data_lag_tolerance=price_delay_tolerance,
    )

    try:
        trading_pair = data_universe.pairs.get_pair_by_human_description((ChainId.polygon, None, reserve.asset_symbol, quote_token))
        exchange = data_universe.pairs.get_exchange_for_pair(trading_pair)
        trading_pair_label = f"{trading_pair.base_token_symbol}-{trading_pair.quote_token_symbol} at {trading_pair.fee} BPS fee tier on {exchange.name}"
        tradeable = trading_pair.is_tradeable()
        trading_pair_link = trading_pair.get_link()

        try:
            price_now, lag = data_universe.candles.get_price_with_tolerance(
                trading_pair,
                now,
                tolerance=price_delay_tolerance,
            )

            price_past, lag = data_universe.candles.get_price_with_tolerance(
                trading_pair,
                past,
                tolerance=price_delay_tolerance,
            )

            change = f"{(price_now - price_past) / price_past * 100:,.2f}"

        except CandleSampleUnavailable as e:
            price_now = "no trades"
            price_past = "no trades"
            change = ""

    except PairNotFoundError as e:
        # Some assets like AAVE do not have USDC quoted pair on Uniswap v3 on Polygon
        trading_pair = None
        trading_pair_label = "No markets available"
        tradeable = False
        price_now = ""
        price_past = ""
        trading_pair_link = ""
        change = ""

    rows.append({
        "Lending asset": reserve.asset_symbol,
        "Leveraged trading": "yes" if (not stablecoin and tradeable) else "no",
        "Stablecoin": "yes" if stablecoin else "no",
        "Trading volume": "yes" if tradeable else "no",
        "Best trading pair": trading_pair_label,
        "Price now (USD)": price_now,
        "Price 30d ago (USD)": price_past,
        "Price %": change,
        "Supply APR": f"{supply_apr_now * 100:,.2f}",
        "Borrow APR": f"{borrow_apr_now * 100:,.2f}",
        "Price data page": trading_pair_link,
        "Lending rate page": lending_link,
    })

df = pd.DataFrame(rows)
df = format_links_for_html_output(df, ("Price data page", "Lending rate page",))

with pd.option_context('display.max_rows', 100):
    display(HTML(df.to_html(escape=False)))


We have HTML output


,Lending asset,Leveraged trading,Stablecoin,Trading volume,Best trading pair,Price now (USD),Price 30d ago (USD),Price %,Supply APR,Borrow APR,Price data page,Lending rate page
0,AAVE,no,no,no,AAVE-USDC at 30 BPS fee tier on Quickswap,67.253261,55.119519,22.01,0.00,0.00,"<a target=""_blank"" href=""https://tradingstrate...","<a target=""_blank"" href=""https://tradingstrate..."
1,agEUR,no,yes,no,No markets available,,,,399.06,619.01,"<a target=""_blank"" href="""">View</a>","<a target=""_blank"" href=""https://tradingstrate..."
2,BAL,no,no,no,BAL-USDC at 30 BPS fee tier on Uniswap v3,no trades,no trades,,499.57,"1,584.08","<a target=""_blank"" href=""https://tradingstrate...","<a target=""_blank"" href=""https://tradingstrate..."
3,CRV,no,no,no,CRV-USDC at 30 BPS fee tier on Quickswap,0.470435,0.446803,5.29,32.73,488.87,"<a target=""_blank"" href=""https://tradingstrate...","<a target=""_blank"" href=""https://tradingstrate..."
4,DAI,no,yes,no,No markets available,,,,290.18,398.55,"<a target=""_blank"" href="""">View</a>","<a target=""_blank"" href=""https://tradingstrate..."
5,DPI,no,no,no,DPI-USDC at 30 BPS fee tier on Quickswap,no trades,no trades,,43.28,384.64,"<a target=""_blank"" href=""https://tradingstrate...","<a target=""_blank"" href=""https://tradingstrate..."
6,EURS,no,yes,no,No markets available,,,,257.05,451.88,"<a target=""_blank"" href="""">View</a>","<a target=""_blank"" href=""https://tradingstrate..."
7,GHST,no,yes,no,No markets available,,,,0.14,18.63,"<a target=""_blank"" href="""">View</a>","<a target=""_blank"" href=""https://tradingstrate..."
8,jEUR,no,yes,no,No markets available,,,,88.13,221.27,"<a target=""_blank"" href="""">View</a>","<a target=""_blank"" href=""https://tradingstrate..."
9,LINK,yes,no,yes,LINK-USDC at 5 BPS fee tier on Uniswap v3,7.493245,6.289054,19.15,4.26,91.00,"<a target=""_blank"" href=""https://tradingstrate...","<a target=""_blank"" href=""https://tradingstrate..."
